<a href="https://colab.research.google.com/github/Shashi1729/Project-4/blob/main/SEM_6_Reserving_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 15.6 MB/s eta 0:00:00


In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import warnings

In [4]:
#excel_file = '/content/drive/MyDrive/Shashi/medmal_pos.csv'
excel_file = '/content/drive/MyDrive/Shashi/prodliab_pos (1).csv'
df = pd.read_csv(excel_file)
df.tail(10)

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_R1,CumPaidLoss_R1,BulkLoss_R1,EarnedPremDIR_R1,EarnedPremCeded_R1,EarnedPremNet_R1,Single,PostedReserve97_R1
6990,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,1997,1,0,0,0,0,0,0,1,110
6991,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,1998,2,0,0,0,0,0,0,1,110
6992,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,1999,3,0,0,0,0,0,0,1,110
6993,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,2000,4,0,0,0,0,0,0,1,110
6994,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,2001,5,0,0,0,0,0,0,1,110
6995,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,2002,6,0,0,0,0,0,0,1,110
6996,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,2003,7,0,0,0,0,0,0,1,110
6997,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,2004,8,0,0,0,0,0,0,1,110
6998,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,2005,9,0,0,0,0,0,0,1,110
6999,44091,Dowa Fire & Marine Ins Co Ltd Us Br,1997,2006,10,0,0,0,0,0,0,1,110


In [5]:
class Triangle:
  def __init__(self,filename):
    self.filename = filename

  def DataFrame(self):
    df = pd.read_csv(self.filename)
    return df

  def column_names(self):
    df = self.DataFrame()
    columns = df.columns.tolist()
    return columns

  def _get_data(self,cond):
    if not cond.empty:  # Check if the condition is not empty
        value = cond.iloc[-1]  # Get the last element of the condition
        return value
    else:
        return 0

  def sum_List(self,L):
    sum = 0
    for i in range(len(L)):
      sum += L[i]
    return sum

  def Create_Dev_Trngl(self,grcode):
    df = self.DataFrame()
    DF = df[df['GRCODE'] == grcode]
    columns = self.column_names()
    DT = []
    for i in range(1988,1998):
      L = []
      for j in range(i,1998):
        condition = (DF['AccidentYear'] == i) & (DF['DevelopmentYear'] == j)
        selected_value = self._get_data(DF.loc[condition, columns[6]])
        L.append(selected_value)
      DT.append(L)
    return DT

  def Create_Vald_Trngl(self,grcode):
    df = self.DataFrame()
    DF = df[df['GRCODE'] == grcode]
    columns = self.column_names()
    DT = []
    for i in range(1988,1998):
      L = []
      for j in range(i,i+10):
        condition = (DF['AccidentYear'] == i) & (DF['DevelopmentYear'] == j)
        selected_value = self._get_data(DF.loc[condition, columns[6]])
        L.append(selected_value)
      DT.append(L);
    return DT

  def Loss_Dvlp_Fctrs(self,grcode):
    T = self.Create_Dev_Trngl(grcode)
    LDF = []
    for i in range(len(T)-1):
      P = []
      for j in range(len(T[i])-1):
        if (T[i][j] != 0):
          f = round(T[i][j+1]/T[i][j],4)
        else:
          f = 1
        P.append(f)
      LDF.append(P)
    return LDF

  def Loss_Dvlp_val(self,grcode):
    T = self.Create_Vald_Trngl(grcode)
    LDF = []
    for i in range(len(T)):
      P = []
      for j in range(len(T[i])-1):
        if (T[i][j] != 0):
          f = round(T[i][j+1]/T[i][j],4)
        else:
          f = 1
        P.append(f)
      LDF.append(P)
    return LDF

  def Display_Loss_Trngl(self,grcode):
    T = self.Create_Dev_Trngl(grcode)
    Accident_Year = [1988+i for i in range(10)]
    col = [12*(i+1) for i in range(10)]
    DF = pd.DataFrame.from_records(T,columns = col, index = Accident_Year)
    print(DF)

  def Display_Factrs_Trngl(self,grcode):  # Give the input as the value returned by Loss_Dvlp_Fctrs
    T = self.Loss_Dvlp_Fctrs(grcode)
    Accident_Year = [1988+i for i in range(9)]
    col = [(12*(i+1),12*(i+2)) for i in range(9)]
    DF = pd.DataFrame.from_records(T,columns = col, index = Accident_Year)
    print(DF)

  def Display_Factrs_Val(self,grcode):  # Give the input as the value returned by Loss_Dvlp_Fctrs
    T = self.Loss_Dvlp_val(self,grcode)
    Accident_Year = [1988+i for i in range(10)]
    col = [(12*(i+1),12*(i+2)) for i in range(10)]
    DF = pd.DataFrame.from_records(T,columns = col, index = Accident_Year)
    print(DF)

  def True_Ultimate_Claims(self,grcode):
    df = self.DataFrame()
    DF = df[df['GRCODE'] == grcode]
    columns = self.column_names()
    L = []
    for i in range(1988,1998):
      condition = (DF['AccidentYear'] == i) & (DF['DevelopmentYear'] == i+9)
      selected_value = self._get_data(DF.loc[condition, columns[6]])
      L.append(selected_value)
    return L

class Dev_Factors:
  def __init__(self,triangle):
    self.triangle = triangle

  def Simple_Avg(sefl,L):
    sum = 0;
    for i in range(len(L)):
      sum += L[i]
    return sum/len(L)

  def Average_LDF(self,grcode):
    LDF = self.triangle.Loss_Dvlp_Fctrs(grcode)
    count = 0
    ADF = []
    while (count < len(LDF)):
      sum = 0
      L = []
      for i in range(len(LDF)-count):
        L.append(LDF[i][count])
      avg = self.Simple_Avg(L)
      ADF.append(avg)
      count += 1
    return ADF

  def Cumulative_LDF(self,grcode):
    ADF = self.Average_LDF(grcode)
    factor = 1
    CDF = [1]
    i = 0; #iterator
    while (i < len(ADF)):
      CDF.insert(0,factor * ADF[-(i+1)])
      factor = factor * ADF[-(i+1)]
      i += 1
    return CDF

class Reserving_mthds:
  def __init__(self,triangle,dfactors):
    self.triangle = triangle
    self.dfactors = dfactors

  def List_grcode(self):
    df = self.triangle.DataFrame()
    L = list(df['GRCODE'].unique())
    return L

  def PREMIUM(self,grcode):
    df = self.triangle.DataFrame()
    New_df = df[df['GRCODE'] == grcode]
    columns = self.triangle.column_names()
    i = 0
    L = []
    while (i < len(New_df)):
      L.append(New_df[columns[10]].tolist()[i])
      i = i + 10
    return L

  def _Div_Lists(self,M,L):
    P = []
    if len(M) == len(L):
       for i in range(len(M)):
         if L[i] != 0:
            P.append(M[i]/L[i])
         else:
            P.append(0)
    return P

  def Count_Zeroes(self,grcode):
    T = self.triangle.Create_Dev_Trngl(grcode)
    count = 0
    for i in range(len(T)):
      for j in range(len(T[i])):
        if T[i][j] == 0:
          count += 1
    return count

  def Refined_GRCODE(self):
    M = []
    GRCODE = self.List_grcode()
    for i in range(len(GRCODE)):
      if self.Count_Zeroes(GRCODE[i]) < 30:
        M.append(GRCODE[i])
    return M

  def Ultimate_Claims(self,grcode):
    LT = self.triangle.Create_Dev_Trngl(grcode)
    CDF = self.dfactors.Cumulative_LDF(grcode)
    UC = []
    for i in range(len(LT)):
      UC.append(LT[i][-1] * CDF[-(i+1)])
    return UC

  def GLM_Ult_Claims(self,grcode):
    L_T = []
    L_T = self.triangle.Create_Dev_Trngl(grcode)
    for j in range(1,len(L_T)):
      # Ignore divide by zero warning
      warnings.filterwarnings("ignore", category=RuntimeWarning)
      L_x = np.array([L_T[i][j-1] if L_T[i][j-1] != 0 else 1 for i in range(len(L_T[0:len(L_T)-j]))])
      L_y = np.array([L_T[i][j] if L_T[i][j] != 0 else 1 for i in range(len(L_T[0:len(L_T)-j]))])
      L_test = np.array([L_T[-i][j-1] if L_T[-i][j-1] != 0 else 1 for i in range(1,(j+1))])

      # Creating and fitting the GLM model
      model = sm.GLM(L_y, L_x, family=sm.families.Gaussian())
      result = model.fit(method = 'bfgs')

      Y_test = result.predict(L_test)
      for k in range(0,j):
        L_T[-(k+1)].append(Y_test[k])

    U = []
    for i in range(len(L_T)):
      U.append(L_T[i][len(L_T[i])-1])
    return U

  def Expected_Claims(self,grcode):
    U_C = self.Ultimate_Claims(grcode)
    prem = self.PREMIUM(grcode)
    L_R = self._Div_Lists(U_C,prem)
    ECR = self.triangle.sum_List(L_R) / len(L_R)
    Expected_UC = np.array(U_C) * np.array([ECR for i in range(len(U_C))])
    return Expected_UC

  def BF_Claims(self,grcode):
    U_C = self.Ultimate_Claims(grcode)
    CDF = self.dfactors.Cumulative_LDF(grcode)
    Exp_Claims = self.Expected_Claims(grcode)
    CDF.reverse()
    Unpaid_Per = np.array([1 - (1 / CDF[i]) if CDF[i] != 0 else 0 for i in range(len(CDF))])
    Paid_Per = np.array([(1 / CDF[i]) if CDF[i] != 0 else 0 for i in range(len(CDF))])
    BF_UC = Paid_Per * U_C + Unpaid_Per * Exp_Claims
    return BF_UC

class DashApp:
    def __init__(self, triangle, Reserving_mthds):
        self.app = dash.Dash(__name__)
        self.triangle = triangle
        self.Reserving_mthds = Reserving_mthds
        self.define_layout()
        self.register_callbacks()

    def define_layout(self):
        U_GRCODE = self.Reserving_mthds.Refined_GRCODE()
        self.app.layout = html.Div([
                          html.H1("Loss Triangle Reserving Interface",style={'backgroundColor':'blue'}),
                          dcc.RadioItems(
                          id = 'radio-items',
                          options = [
                                     {'label':'Ultimate Claims', 'value':'Ultimate Claims'},
                                     {'label':'Mean Squared Error', 'value':'Mean Squared Error'}
                                    ],
                          value = 'Ultimate Claims',
                          labelStyle = {'display':'block'}
                                    ),
                          html.Br(),
                          dcc.Dropdown(
                                       id='dropdown',
                                       #options=[{'label':'669','value':669},{'label':'683','value':683}],
                                       options=[{'label':'{:d}'.format(x),'value':x} for x in U_GRCODE],
                                       value=U_GRCODE[0],
                                       multi=False,
                                       style={'width': '50%', 'marginBottom': '20px'}
                                      ),
                          dcc.Graph(id='output-graph')
                          ])

    def register_callbacks(self):
        @self.app.callback(
        Output('output-graph','figure'),
        [Input('dropdown', 'value'), Input('radio-items', 'value')]
        )
        def update_plot(grcode, radio_value):
            grcode = int(grcode)
            U_C = self.Reserving_mthds.Ultimate_Claims(grcode)
            Exp_UC = self.Reserving_mthds.Expected_Claims(grcode)
            BF_UC = self.Reserving_mthds.BF_Claims(grcode)
            GLM_UC = self.Reserving_mthds.GLM_Ult_Claims(grcode)
            if radio_value == 'Ultimate Claims':
               L = []
               Tr_UC = self.triangle.True_Ultimate_Claims(grcode)
               L.append(self.triangle.sum_List(Tr_UC))
               L.append(self.triangle.sum_List(U_C))
               L.append(self.triangle.sum_List(Exp_UC))
               L.append(self.triangle.sum_List(BF_UC))
               L.append(self.triangle.sum_List(GLM_UC))
               #L.append(U_C_GLM)
               T_DATA = {'Reserving-method': ['True_Ultimate_Claims', 'Dev_UC', 'Sum_Expected_UC', 'Sum_BF_UC', 'GLM_UC'],
                         'Ultimate_Claims': L}
               DF = pd.DataFrame(T_DATA)
               fig = px.bar(DF, x='Reserving-method', y='Ultimate_Claims',
                         title=f'Company: {grcode}',
                         labels={'Ultimate_Claims': 'Amount'})
               return fig
            else:
               M = []
               M.append(mean_squared_error(self.triangle.True_Ultimate_Claims(grcode), U_C))
               M.append(mean_squared_error(self.triangle.True_Ultimate_Claims(grcode), Exp_UC))
               M.append(mean_squared_error(self.triangle.True_Ultimate_Claims(grcode), BF_UC))
               M.append(mean_squared_error(self.triangle.True_Ultimate_Claims(grcode), GLM_UC))
               T_DATA = {'Mean_Squared_Error': ['Dev_mse', 'Exp_mse', 'BF_mse', 'GLM_mse'],
                         'Ultimate_Claims': M}
               DF = pd.DataFrame(T_DATA)
               fig = T_DATA = {'Mean_Squared_Error': ['Dev_mse', 'Exp_mse', 'BF_mse', 'GLM_mse'],
                         'mse': M}
               DF = pd.DataFrame(T_DATA)
               fig = px.line(DF, x='Mean_Squared_Error', y='mse',
                            title=f'Company: {grcode}',
                            labels={'mse': 'estimate'})
               return fig

    def run(self):
        self.app.run_server(debug=True)



In [6]:
Tr = Triangle(excel_file)
D_f = Dev_Factors(Tr)
R_m = Reserving_mthds(Tr, D_f)

if __name__ == '__main__':
    dash_app = DashApp(Tr,R_m)
    dash_app.run()

<IPython.core.display.Javascript object>